## Registration Information:
- **Registration Number:** RA2111003010596
- **Name:** Pulkit Shringi
- **Section:** I1
- **Email:** ph1973@srmist.edu.in


## 1. Data regions and Data clauses:

In [ ]:
import numpy as np
from numba import cuda

# Define kernel function
@cuda.jit
def add_arrays(a, b, c):
    idx = cuda.grid(1)
    if idx < len(a):
        c[idx] = a[idx] + b[idx]

# Main function
def main():
    # Array size
    N = 1000

    # Initialize input arrays on the CPU
    a = np.random.rand(N).astype(np.float32)
    b = np.random.rand(N).astype(np.float32)

    # Allocate output array on the CPU
    c = np.zeros_like(a)

    # Allocate memory on the GPU and transfer data
    d_a = cuda.to_device(a)
    d_b = cuda.to_device(b)
    d_c = cuda.to_device(c)

    # Launch kernel with data regions and data clauses
    with cuda.gpus[0]:
        add_arrays[N, 1](d_a, d_b, d_c)

    # Transfer result back to the CPU
    d_c.copy_to_host(c)

    # Print result
    print("Result array (c):", c)

if __name__ == "__main__":
    main()


Result array (c): [0.6096229  0.8598883  0.5664964  0.34593064 1.0582803  1.0863649
 1.4230244  1.277323   0.9146615  1.4402969  1.2720772  0.5846211
 0.9630954  1.7520881  1.8162138  1.2655301  0.40332395 0.29248813
 0.9078927  1.0974655  1.2489278  1.5679469  1.0475826  0.9132792
 0.7656118  1.5614353  0.921546   1.4091947  1.6545918  1.1901544
 1.1047037  1.4551964  0.5218992  0.37507898 0.14032298 0.4797646
 0.8149321  1.758665   1.0074725  0.6575542  0.6529774  0.79849875
 1.2796831  0.9507731  0.6630018  1.1136363  1.4834213  1.0963898
 0.08343747 1.6607887  1.1754173  0.5026342  1.3812509  1.3086209
 1.1488967  1.650713   0.2926948  1.7733827  1.2763078  0.6931138
 0.30873072 1.2590013  0.9102328  0.7219488  1.0180261  0.7449136
 0.6055236  1.4622303  1.4683558  1.2637503  1.3511887  0.812295
 0.95701796 0.5314009  1.184686   0.9493573  0.9276598  0.7817453
 1.4221996  0.9534637  1.1939527  1.4777131  1.0108751  1.3441068
 0.5606459  1.491529   0.8740469  0.40571907 0.5913063  0

## Importing Libraries:
- `numpy` is imported to work with arrays in Python.
- `numba.cuda` is imported to utilize CUDA for parallel computing.

## Kernel Function Definition (`add_arrays`):
- This is a function decorated with `@cuda.jit`, indicating that it's a CUDA kernel.
- It takes three arguments: `a`, `b`, and `c`.
- Inside the kernel function, it computes the element-wise sum of arrays `a` and `b` and stores the result in array `c`.
- `cuda.grid(1)` returns the thread index within a one-dimensional block, which is then used to determine which element of the arrays each thread will process.
- The `if` condition ensures that only threads within the array bounds participate in computation.

## Main Function:
- `main()` function initializes arrays `a` and `b` with random float values using NumPy.
- It allocates memory for array `c` on the CPU using `np.zeros_like(a)` which creates an array of zeros with the same shape and type as `a`.
- Memory for arrays `a`, `b`, and `c` is allocated on the GPU using `cuda.to_device()`, which copies the arrays from the CPU to the GPU.
- The kernel function `add_arrays` is launched on the GPU using `cuda.gpus[0]` to select the first GPU. The `add_arrays` kernel is executed with `N` threads, each processing one element of the arrays.
- After the kernel execution, the result array `c` is copied back from the GPU to the CPU using `d_c.copy_to_host(c)`.
- Finally, the result array `c` is printed.


## 2. Reduction Clause:

In [ ]:
import numpy as np
from numba import cuda

# Define the kernel function with OpenACC pragma for parallel execution
@cuda.jit
def sum_reduction(a, result):
    tid = cuda.threadIdx.x
    block_size = cuda.blockDim.x
    grid_size = cuda.gridDim.x
    stride = block_size * grid_size

    # Shared memory for block-level reduction
    shared_sum = cuda.shared.array(256, dtype=np.float32)

    # Perform the reduction within each block
    local_sum = 0
    for i in range(tid, len(a), stride):
        local_sum += a[i]

    # Store the local sum into shared memory
    shared_sum[tid] = local_sum
    cuda.syncthreads()

    # Perform block-level reduction using shared memory
    for s in range(block_size // 2):
        idx = 2 * s * tid
        if idx < block_size:
            shared_sum[idx] += shared_sum[idx + 1]
        cuda.syncthreads()

    # Store the block-level reduction result to global memory
    if tid == 0:
        result[0] = shared_sum[0]

# Main function
def main():
    # Array size
    N = 10000
    # Initialize array with random values
    a = np.random.rand(N).astype(np.float32)

    # Allocate memory on the GPU
    d_a = cuda.to_device(a)
    result = cuda.to_device(np.array([0], dtype=np.float32))

    # Define grid and block dimensions
    block_dim = 256
    grid_dim = (N + block_dim - 1) // block_dim

    # Launch kernel
    sum_reduction[grid_dim, block_dim](d_a, result)

    # Copy result back to host
    cuda.synchronize()
    final_result = result.copy_to_host()[0]

    print("Sum of array elements:", final_result)

if __name__ == "__main__":
    main()


Sum of array elements: 43.49064


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 40 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))



### Importing Libraries:
- `numpy` is imported to work with arrays in Python.
- `numba.cuda` is imported to utilize CUDA for parallel computing.

### Kernel Function Definition (`sum_reduction`):
- This kernel function performs a sum reduction on an input array `a`.
- It utilizes CUDA for parallel execution.
- It takes two arguments: `a` (the input array) and `result` (an array to store the final result).
- The function is decorated with `@cuda.jit`, indicating it's a CUDA kernel.
- Inside the kernel:
  - It retrieves the thread ID, block size, grid size, and stride.
  - Shared memory `shared_sum` is allocated for block-level reduction.
  - Each thread calculates a local sum within its block.
  - The local sums are stored in shared memory.
  - Block-level reduction is performed using shared memory.
  - The final block-level reduction result is stored in global memory.

### Main Function (`main()`):
- The main function initializes the array `a` with random float values.
- Memory for array `a` is allocated on the GPU using `cuda.to_device()`.
- Memory for the result is also allocated on the GPU.
- Grid and block dimensions are defined to launch the kernel with appropriate parallelization.
- The kernel `sum_reduction` is launched on the GPU.
- The result is copied back to the CPU.
- The final result, which is the sum of array elements, is printed.

### Execution:
- When executed, this code performs a parallel sum reduction of an array on the GPU using CUDA.
- It utilizes shared memory and block-level reduction to efficiently compute the sum.
- The result is then copied back to the CPU for further processing or display.



## 3. Loop Optimization:

In [ ]:
import numpy as np
from numba import cuda

# Define kernel function with loop optimizations
@cuda.jit
def square_elements(a, b):
    idx = cuda.grid(1)
    if idx < len(a):
        # Perform loop optimizations (e.g., loop unrolling)
        sum = 0
        for i in range(10):
            sum += a[idx] * a[idx]
        b[idx] = sum

# Main function
def main():
    # Array size
    N = 1000

    # Initialize input array on the CPU
    a = np.random.rand(N).astype(np.float32)

    # Allocate output array on the CPU
    b = np.zeros_like(a)

    # Allocate memory on the GPU and transfer data
    d_a = cuda.to_device(a)
    d_b = cuda.to_device(b)

    # Launch kernel with data regions and loop optimizations
    with cuda.gpus[0]:
        square_elements[N, 1](d_a, d_b)

    # Transfer result back to the CPU
    d_b.copy_to_host(b)

    # Print result
    print("Result array (b):", b)

if __name__ == "__main__":
    main()


Result array (b): [2.82750368e-01 6.51936102e+00 6.88080788e+00 6.67099905e+00
 9.01152611e+00 5.52901983e+00 9.48503399e+00 1.62459326e+00
 1.34035540e+00 3.38841408e-01 2.54158401e+00 6.42568636e+00
 8.11347961e-02 6.53884125e+00 1.23186886e+00 1.88536298e+00
 2.01648474e-02 3.28639960e+00 1.32596266e+00 9.43049622e+00
 3.63169275e-02 2.41000915e+00 1.66276157e+00 8.99422228e-01
 3.10910165e-01 9.41649377e-02 3.11517024e+00 1.87472641e+00
 1.32113855e-05 7.61527634e+00 1.18282318e-01 6.92205317e-03
 2.62890625e+00 2.87200958e-01 9.36566448e+00 7.88177776e+00
 3.51897240e+00 4.12872696e+00 5.47927558e-01 1.83146477e+00
 6.52820396e+00 9.02347453e-03 6.22714832e-02 3.77670586e-01
 6.32136297e+00 7.84441009e-02 6.08687305e+00 3.45068127e-02
 5.08125830e+00 1.60449088e+00 2.26710439e-02 3.59896161e-02
 2.26083064e+00 4.42354298e+00 1.10703254e+00 4.73594379e+00
 1.48112357e+00 1.62149024e+00 4.34323400e-01 5.71614027e+00
 5.88607740e+00 7.48338938e-01 8.52745771e-01 5.66099119e+00
 8.365


### Importing Libraries:
- `numpy` is imported to work with arrays in Python.
- `numba.cuda` is imported to utilize CUDA for parallel computing.

### Kernel Function Definition (`square_elements`):
- This kernel function calculates the square of each element in the input array `a` and stores the result in array `b`.
- It utilizes CUDA for parallel execution.
- It is decorated with `@cuda.jit`, indicating it's a CUDA kernel.
- Inside the kernel:
  - `cuda.grid(1)` is used to retrieve the thread index within a one-dimensional block.
  - Each thread calculates the square of the corresponding element in array `a`.
  - The result is stored in the corresponding position in array `b`.
  - Loop optimizations, such as loop unrolling, are applied for better performance.

### Main Function (`main()`):
- The main function initializes the input array `a` with random float values.
- Memory for the output array `b` is allocated on the CPU using `np.zeros_like(a)`.
- Memory for arrays `a` and `b` is allocated on the GPU using `cuda.to_device()`.
- The kernel `square_elements` is launched on the GPU.
- The result is copied back to the CPU.
- The resulting array `b` containing the squares of elements is printed.

### Execution:
- When executed, this code performs parallel computation of the squares of elements in an array on the GPU using CUDA.
- Loop optimizations are applied within the kernel for better performance.
- The result is then copied back to the CPU for further processing or display.



## 4. Parallel and kernel directives:

In [ ]:
import numpy as np
from numba import cuda

# Define kernel function
@cuda.jit
def multiply_elements(a, b, c):
    idx = cuda.grid(1)
    if idx < len(a):
        c[idx] = a[idx] * b[idx]

# Main function
def main():
    # Array size
    N = 1000

    # Initialize input arrays on the CPU
    a = np.random.rand(N).astype(np.float32)
    b = np.random.rand(N).astype(np.float32)

    # Allocate output array on the CPU
    c = np.zeros_like(a)

    # Allocate memory on the GPU
    d_a = cuda.to_device(a)
    d_b = cuda.to_device(b)
    d_c = cuda.to_device(c)

    # Launch kernel with parallel directive
    block_size = 256
    grid_size = (N + block_size - 1) // block_size

    multiply_elements[grid_size, block_size](d_a, d_b, d_c)

    # Transfer result back to the CPU
    d_c.copy_to_host(c)

    # Print result
    print("Result array (c):", c)

if __name__ == "__main__":
    main()


Result array (c): [2.02094227e-01 9.43410993e-02 3.86913896e-01 1.29064202e-01
 3.20593596e-01 1.97348729e-01 5.85442558e-02 4.25929189e-01
 1.12647630e-01 1.34312719e-01 1.18314624e-01 2.98026875e-02
 4.59079415e-01 1.61380306e-01 8.54636490e-01 6.75200969e-02
 2.06291750e-01 3.04460436e-01 2.11886689e-01 2.86172032e-01
 1.43993109e-01 2.71580607e-01 8.72256100e-01 1.35649502e-01
 4.62028027e-01 4.29785103e-02 3.25537324e-01 3.47620100e-02
 7.97379836e-02 3.81804496e-01 2.35465318e-01 1.07487142e-01
 1.49909809e-01 9.51573700e-02 1.49219945e-01 1.07929312e-01
 2.25038137e-02 1.93623245e-01 4.25525963e-01 1.02540612e-01
 2.48345688e-01 1.74852088e-01 1.65970298e-03 3.24685693e-01
 6.16362274e-01 5.21901786e-01 7.65474975e-01 1.69166595e-01
 7.24604607e-01 1.88321061e-02 2.49669120e-01 9.35589373e-02
 3.98936689e-01 3.46934438e-01 1.38679564e-01 2.62115672e-02
 1.49742872e-01 1.71001013e-02 1.17050640e-01 7.26336613e-03
 1.96498990e-01 1.78179401e-03 1.03543773e-01 1.06195807e-01
 5.135

/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:536: NumbaPerformanceWarning: Grid size 4 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


### Importing Libraries:
- `numpy` is imported to work with arrays in Python.
- `numba.cuda` is imported to utilize CUDA for parallel computing.

### Kernel Function Definition (`multiply_elements`):
- This kernel function computes the element-wise product of two input arrays `a` and `b` and stores the result in array `c`.
- It is decorated with `@cuda.jit`, indicating it's a CUDA kernel.
- Inside the kernel:
  - `cuda.grid(1)` is used to retrieve the thread index within a one-dimensional block.
  - Each thread calculates the product of corresponding elements in arrays `a` and `b`.
  - The result is stored in the corresponding position in array `c`.

### Main Function (`main()`):
- The main function initializes two input arrays `a` and `b` with random float values.
- Memory for the output array `c` is allocated on the CPU using `np.zeros_like(a)`.
- Memory for arrays `a`, `b`, and `c` is allocated on the GPU using `cuda.to_device()`.
- The grid and block sizes are calculated to determine the number of threads and blocks to launch the kernel.
- The kernel `multiply_elements` is launched on the GPU.
- The result is copied back to the CPU.
- The resulting array `c` containing the element-wise product of `a` and `b` is printed.

### Execution:
- When executed, this code performs parallel computation of the element-wise product of two arrays on the GPU using CUDA.
- It efficiently utilizes parallelism provided by CUDA to perform computations faster compared to traditional CPU-based methods.
- The result is then copied back to the CPU for further processing or display.

